In [ ]:
# 仮想環境内でパッケージをインストール
import sys
!{sys.executable} -m pip install azure-ai-agents

In [ ]:
from dotenv import load_dotenv

load_dotenv('../concierge-agent/.env')

In [ ]:
import os
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ConnectedAgentTool, AgentThreadCreationOptions, ListSortOrder
from azure.identity import DefaultAzureCredential


agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_CONNECTION_STRING"],
    credential=DefaultAzureCredential(),
)

In [ ]:
stock_price_agent = agents_client.create_agent(
    model=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],
    name="stock_price_bot",
    instructions="Your job is to get the stock price of a company. If you don't know the realtime stock price, return the last known stock price.",
    #tools=... # tools to help the agent get stock prices
)

In [ ]:
connected_agent = ConnectedAgentTool(
    id=stock_price_agent.id, name="connected_agent", description="Gets the stock price of a company"
)

In [ ]:
agent = agents_client.create_agent(
    model=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are a helpful agent, and use the available tools to get stock prices.",
    tools=connected_agent.definitions,
)

print(f"Created agent, ID: {agent.id}")

thread_run = agents_client.create_thread_and_process_run(
    agent_id=agent.id,
    thread=AgentThreadCreationOptions(
        messages=[
            {
                "role": "user",
                "content": "What is the stock price of Microsoft?",
            }
        ]
    ),
)
print(f"Created thread run, ID: {thread_run.id}")
print(f"Thread run status: {thread_run.status}")

messages = agents_client.messages.list(thread_id=thread_run.thread_id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

In [ ]:
agent = agents_client.create_agent(
    model=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are a helpful agent, and use the available tools to get stock prices.",
    tools=connected_agent.definitions,
)

print(f"Created agent, ID: {agent.id}")

run = agents_client.create_thread_and_run(
    agent_id=agent.id,
    thread=AgentThreadCreationOptions(
        messages=[
            {
                "role": "user",
                "content": "What is the stock price of Microsoft?",
            }
        ]
    ),
)

print(f"Created thread run, ID: {run.id}")
print(f"Thread run status: {run.status}")

messages = agents_client.messages.list(thread_id=run.thread_id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")


# Clean up resources
agents_client.delete_agent(agent.id)

### AI Foundry SDK から呼び出す

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ConnectedAgentTool

# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_CONNECTION_STRING"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

connected_agent = ConnectedAgentTool(
    id=stock_price_agent.id, name="connected_agent", description="Gets the stock price of a company"
)

with project_client:
    # Create an agent with the Bing Grounding tool
    agent = project_client.agents.create_agent(
        model=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        tools=connected_agent.definitions,  # Attach the tool
    )
    print(f"Created agent, ID: {agent.id}")

    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content="What is the weather in Seattle today?",  # Message content
    )
    print(f"Created message, ID: {message['id']}")

    # Create and process an agent run
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check if the run failed
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Fetch and log all messages
    messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for message in messages:
        print(f"Role: {message.role}, Content: {message.content}")

    # Delete the agent when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")